In [1]:
import rasterio as rio

In [2]:
granule_loc = 'D:/canopy_data/error_fixing/granule.tif'

src = rio.open(granule_loc)

In [3]:
src.meta

{'driver': 'GTiff',
 'dtype': 'uint16',
 'nodata': None,
 'width': 3340,
 'height': 3340,
 'count': 17,
 'crs': CRS.from_epsg(4326),
 'transform': Affine(8.983152841195215e-05, 0.0, 19.58929190620917,
        0.0, -8.983152841195215e-05, -0.40954193803008987)}

In [4]:
src.profile

{'driver': 'GTiff', 'dtype': 'uint16', 'nodata': None, 'width': 3340, 'height': 3340, 'count': 17, 'crs': CRS.from_epsg(4326), 'transform': Affine(8.983152841195215e-05, 0.0, 19.58929190620917,
       0.0, -8.983152841195215e-05, -0.40954193803008987), 'blockxsize': 256, 'blockysize': 256, 'tiled': True, 'compress': 'lzw', 'interleave': 'pixel'}

In [1]:
from convert_rasters import convert_granule

ModuleNotFoundError: No module named 'rasterio'

In [ ]:
granule_path = 'enter_here'

dest_dir = 's3://canopy-production-ml/full_congo_basin/test_conversion/'

result = convert_granule(granule_path, dest_dir)

result

In [67]:
import json


base_command = [
    'aws',
    'lambda',
    'invoke',
    '--function-name',
    'cb-raster-process',
    '--cli-binary-format',
    'raw-in-base64-out',
    '--payload',
    'payload_dict_here',
    'response.json',
]

In [68]:
granule_path = r"s3://canopy-production-ml/full_congo_basin/02.17.21_CB_GEE_Pull/1000_full_congo_export_v12_all_bands_Feb_11_12_22_04_2021.tif"

payload_dict = {'granule_path': granule_path, 'dest_dir': r"s3://canopy-production-ml/full_congo_basin/RGB/"}

In [69]:
test_command = base_command.copy()

test_command[8] = json.dumps(payload_dict)

In [70]:
test_command

['aws',
 'lambda',
 'invoke',
 '--function-name',
 'cb-raster-process',
 '--cli-binary-format',
 'raw-in-base64-out',
 '--payload',
 '{"granule_path": "s3://canopy-production-ml/full_congo_basin/02.17.21_CB_GEE_Pull/1000_full_congo_export_v12_all_bands_Feb_11_12_22_04_2021.tif", "dest_dir": "s3://canopy-production-ml/full_congo_basin/RGB/"}',
 'response.json']

In [71]:
import subprocess


subprocess.run(test_command, capture_output=True)

CompletedProcess(args=['aws', 'lambda', 'invoke', '--function-name', 'cb-raster-process', '--cli-binary-format', 'raw-in-base64-out', '--payload', '{"granule_path": "s3://canopy-production-ml/full_congo_basin/02.17.21_CB_GEE_Pull/1000_full_congo_export_v12_all_bands_Feb_11_12_22_04_2021.tif", "dest_dir": "s3://canopy-production-ml/full_congo_basin/RGB/"}', 'response.json'], returncode=254, stdout=b'', stderr=b'\r\nAn error occurred (ResourceNotFoundException) when calling the Invoke operation: Function not found: arn:aws:lambda:us-west-1:963659202518:function:cb-raster-process\r\n')

In [73]:
json_file_loc = 'D:/canopy_data/predict_3257-2021-05-18-22-32-24.json'

with open(json_file_loc) as f:
    data = json.load(f)

In [74]:
len(data)

972

In [75]:
list(data.keys())[0]

'681'

In [76]:
data['681']

[{'window_id': 1,
  'polygon_coords': [[-10905062.77726229, -5260429.554581894],
   [-10905062.77726229, -5258613.415615976],
   [-10906878.916228207, -5258613.415615976],
   [-10906878.916228207, -5260429.554581894],
   [-10905062.77726229, -5260429.554581894]],
  'labels': []},
 {'window_id': 2,
  'polygon_coords': [[-10903246.638296371, -5260429.554581894],
   [-10903246.638296371, -5258613.415615976],
   [-10905062.77726229, -5258613.415615976],
   [-10905062.77726229, -5260429.554581894],
   [-10903246.638296371, -5260429.554581894]],
  'labels': []},
 {'window_id': 3,
  'polygon_coords': [[-10901430.499330452, -5260429.554581894],
   [-10901430.499330452, -5258613.415615976],
   [-10903246.638296371, -5258613.415615976],
   [-10903246.638296371, -5260429.554581894],
   [-10901430.499330452, -5260429.554581894]],
  'labels': []},
 {'window_id': 4,
  'polygon_coords': [[-10899614.360364534, -5260429.554581894],
   [-10899614.360364534, -5258613.415615976],
   [-10901430.499330452, 

In [78]:
label_dict = {'null': 0}

for i, k in enumerate(data):
    for window in data[k]:
        labels = window['labels']
        if len(labels) == 0:
            label_dict['null'] += 1
        else:
            for label in labels:
                if label in label_dict:
                    label_dict[label] += 1
                else:
                    label_dict[label] = 1

In [79]:
label_dict

{'null': 1346976,
 'ISL': 66597,
 'Shifting_cultivation': 248700,
 'Industrial_agriculture': 39649,
 'Mining': 1893,
 'Roads': 28011}

In [80]:
json_file_loc = 'D:/canopy_data/predict_3257-2021-05-15-22-18-23.json'

with open(json_file_loc) as f:
    data2 = json.load(f)

In [81]:
data1_granules = list(data.keys())
data2_granules = list(data2.keys())

In [82]:
len(data2_granules)

1022

In [84]:
len(set(data1_granules + data2_granules))

1994